In [48]:
# Homography with mouse click

import cv2
import numpy as np

selected_points = []

# Function to handle mouse clicks
def mouse_callback(event, x, y, flags, param):
    global selected_points
    
    if event == cv2.EVENT_LBUTTONDOWN:
        selected_points.append((x, y))
        cv2.circle(param, (x, y), 5, (0, 255, 0), -1)
        cv2.imshow("Input Image", param)

input_image = cv2.imread("20230403_132407.jpg")

# Resize the input image to fit within the screen resolution
max_width = 800  # Maximum width for display
max_height = 600  # Maximum height for display
aspect_ratio = input_image.shape[1] / input_image.shape[0]
if aspect_ratio > 1:
    display_width = min(max_width, input_image.shape[1])
    display_height = int(display_width / aspect_ratio)
else:
    display_height = min(max_height, input_image.shape[0])
    display_width = int(display_height * aspect_ratio)
resized_image = cv2.resize(input_image, (display_width, display_height))

cv2.imshow("Input Image", resized_image)
cv2.setMouseCallback("Input Image", mouse_callback, resized_image)

# Wait for four points to be selected
while len(selected_points) < 4:
    cv2.waitKey(1)

cv2.destroyAllWindows()

src_points_resized = np.array(selected_points, dtype=np.float32)

print("Display Width = ", display_width)
print("Display Height = ", display_height)
print("Original Width = ", input_image.shape[1])
print("Original Height = ", input_image.shape[0])

print("Selected points:", src_points_resized)

# Scale selected points to represent original coordinates
scale_x = input_image.shape[1] / resized_image.shape[1]
scale_y = input_image.shape[0] / resized_image.shape[0]
src_points = src_points_resized * np.array([scale_x, scale_y])

print("Selected points (original coordinates):", src_points)

dst_points = np.array([[0, 0], [input_image.shape[1], 0], [0, input_image.shape[0]], [input_image.shape[1], input_image.shape[0]]], dtype=np.float32)

homography_matrix, _ = cv2.findHomography(src_points, dst_points)

output_image = cv2.warpPerspective(input_image, homography_matrix, (input_image.shape[1], input_image.shape[0]))

cv2.imwrite("Homography_mouse_selected.jpg", output_image)

# Convert the homography image back to the original format
homography_matrix, _ = cv2.findHomography(dst_points, src_points)

reversed_output_image = cv2.warpPerspective(output_image, homography_matrix, (output_image.shape[1], output_image.shape[0]))

cv2.imwrite("Reversed_homography_mouse_selected.jpg", reversed_output_image)

Display Width =  800
Display Height =  600
Original Width =  4032
Original Height =  3024
Selected points: [[174.  93.]
 [682. 171.]
 [  8. 306.]
 [685. 460.]]
Selected points (original coordinates): [[ 876.96  468.72]
 [3437.28  861.84]
 [  40.32 1542.24]
 [3452.4  2318.4 ]]


True

In [53]:
# Homography with corner detection

import numpy as np
import cv2

input_image = cv2.imread("IMG_20200316_171750.jpg")

# Resize the input image to fit within the screen resolution
max_width = 800  # Maximum width for display
max_height = 600  # Maximum height for display
aspect_ratio = input_image.shape[1] / input_image.shape[0]
if aspect_ratio > 1:
    display_width = min(max_width, input_image.shape[1])
    display_height = int(display_width / aspect_ratio)
else:
    display_height = min(max_height, input_image.shape[0])
    display_width = int(display_height * aspect_ratio)
resized_image = cv2.resize(input_image, (display_width, display_height))

gray = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)

# Detect corners
corners = cv2.goodFeaturesToTrack(gray, 100, 0.01, 10)
corners = np.int0(corners)

leftmost_up = [float('inf'), float('inf')]
leftmost_bottom = [float('inf'), -float('inf')]
rightmost_up = [-float('inf'), float('inf')]
rightmost_bottom = [-float('inf'), -float('inf')]

for corner in corners:
    x, y = corner.ravel()
    
    if y < leftmost_up[1]:
        leftmost_up = [x, y]

    if x < leftmost_bottom[0]:
        leftmost_bottom = [x, y]

    if x > rightmost_up[0]:
        rightmost_up = [x, y]

    if y > rightmost_bottom[1]:
        rightmost_bottom = [x, y]

print("Leftmost Up Corner:", leftmost_up)
print("Leftmost Bottom Corner:", leftmost_bottom)
print("Rightmost Up Corner:", rightmost_up)
print("Rightmost Bottom Corner:", rightmost_bottom)

# Scale selected points to represent original coordinates
scale_x = input_image.shape[1] / resized_image.shape[1]
scale_y = input_image.shape[0] / resized_image.shape[0]

src_points = np.array([leftmost_up, rightmost_up, leftmost_bottom, rightmost_bottom], dtype=np.float32)

src_points = src_points * np.array([scale_x, scale_y])

print("Selected points:", src_points)

dst_points = np.array([[0, 0], [input_image.shape[1], 0], [0, input_image.shape[0]], [input_image.shape[1], input_image.shape[0]]], dtype=np.float32)

print("Destination points:", dst_points)

homography_matrix, _ = cv2.findHomography(src_points, dst_points)

output_image = cv2.warpPerspective(input_image, homography_matrix, (input_image.shape[1], input_image.shape[0]))

cv2.imwrite("Homography_corner_detected.jpg", output_image)

# Convert the homography image back to the original format
homography_matrix, _ = cv2.findHomography(dst_points, src_points)

reversed_output_image = cv2.warpPerspective(output_image, homography_matrix, (output_image.shape[1], output_image.shape[0]))

cv2.imwrite("Reversed_homography_corner_detected.jpg", reversed_output_image)

C:\Users\ekasi\AppData\Local\Temp\ipykernel_314452\522126478.py:24: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  corners = np.int0(corners)


Leftmost Up Corner: [243, 98]
Leftmost Bottom Corner: [88, 278]
Rightmost Up Corner: [689, 228]
Rightmost Bottom Corner: [607, 492]
Selected points: [[1399.68  564.48]
 [3968.64 1313.28]
 [ 506.88 1601.28]
 [3496.32 2833.92]]
Destination points: [[   0.    0.]
 [4608.    0.]
 [   0. 3456.]
 [4608. 3456.]]


True

In [50]:
# Homography with color detection

import cv2
import numpy as np

def get_limits(color):
    c = np.uint8([[color]])  # BGR values
    hsvC = cv2.cvtColor(c, cv2.COLOR_BGR2HSV)

    hue = hsvC[0][0][0]  # Get the hue value

    if hue >= 165:  # Upper limit for divided red hue
        lowerLimit = np.array([hue - 10, 100, 100], dtype=np.uint8)
        upperLimit = np.array([180, 255, 255], dtype=np.uint8)
    elif hue <= 15:  # Lower limit for divided red hue
        lowerLimit = np.array([0, 100, 100], dtype=np.uint8)
        upperLimit = np.array([hue + 10, 255, 255], dtype=np.uint8)
    else:
        lowerLimit = np.array([hue - 10, 100, 100], dtype=np.uint8)
        upperLimit = np.array([hue + 10, 255, 255], dtype=np.uint8)

    return lowerLimit, upperLimit

image = cv2.imread('colored_field.jpg')

hsvImage = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# Colors and their corresponding BGR values
colors = {
    'red': [0, 0, 255],
    'yellow': [0, 255, 255],
    'green': [0, 255, 0],
    'blue': [255, 0, 0]
}

color_locations = {}

for color_name, color_value in colors.items():
    lowerLimit, upperLimit = get_limits(color_value)

    mask = cv2.inRange(hsvImage, lowerLimit, upperLimit)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    color_locations[color_name] = []

    for contour in contours:
        bbox = cv2.boundingRect(contour)
        color_locations[color_name].append(bbox)

src_points = np.array([color_locations['red'][0][:2], color_locations['yellow'][0][:2],
                       color_locations['green'][0][:2], color_locations['blue'][0][:2]], dtype=np.float32)
dst_points = np.array([[0, 0], [image.shape[1], 0], [0, image.shape[0]], [image.shape[1], image.shape[0]]], dtype=np.float32)

homography_matrix, _ = cv2.findHomography(src_points, dst_points)

output_image = cv2.warpPerspective(image, homography_matrix, (image.shape[1], image.shape[0]))

cv2.imwrite("Homography_color_detected.jpg", output_image)



True

In [54]:
# Player detection with mouse click

import cv2
import numpy as np

selected_points = []

# Function to handle mouse clicks
def mouse_callback(event, x, y, flags, param):
    global selected_points
    
    if event == cv2.EVENT_LBUTTONDOWN:
        selected_points.append((x, y))
        cv2.circle(param, (x, y), 5, (0, 255, 0), -1)
        cv2.imshow("Input Image", param)

input_image = cv2.imread("20230403_132407.jpg")

# Resize the input image to fit within the screen resolution
max_width = 800  # Maximum width for display
max_height = 600  # Maximum height for display
aspect_ratio = input_image.shape[1] / input_image.shape[0]
if aspect_ratio > 1:
    display_width = min(max_width, input_image.shape[1])
    display_height = int(display_width / aspect_ratio)
else:
    display_height = min(max_height, input_image.shape[0])
    display_width = int(display_height * aspect_ratio)
resized_image = cv2.resize(input_image, (display_width, display_height))

cv2.imshow("Input Image", resized_image)
cv2.setMouseCallback("Input Image", mouse_callback, resized_image)

# Wait for four points to be selected
while len(selected_points) < 4:
    cv2.waitKey(1)

cv2.destroyAllWindows()

src_points_resized = np.array(selected_points, dtype=np.float32)

print("Display Width = ", display_width)
print("Display Height = ", display_height)
print("Original Width = ", input_image.shape[1])
print("Original Height = ", input_image.shape[0])

print("Selected points:", src_points_resized)

# Scale selected points to represent original coordinates
scale_x = input_image.shape[1] / resized_image.shape[1]
scale_y = input_image.shape[0] / resized_image.shape[0]
src_points = src_points_resized * np.array([scale_x, scale_y])

print("Selected points (original coordinates):", src_points)

dst_points = np.array([[0, 0], [input_image.shape[1], 0], [0, input_image.shape[0]], [input_image.shape[1], input_image.shape[0]]], dtype=np.float32)

homography_matrix, _ = cv2.findHomography(src_points, dst_points)

output_image = cv2.warpPerspective(input_image, homography_matrix, (input_image.shape[1], input_image.shape[0]))

cv2.imwrite("Player_detection_first_homography.jpg", output_image)

# Function to check if a point is too close to any of the existing circles
def eliminate_close_features(new_circle_center, existing_circle_centers, min_distance):
    for center in existing_circle_centers:
        if np.linalg.norm(np.array(new_circle_center) - np.array(center)) < min_distance:
            return True
    return False


# Convert the image to HSV color space
hsv = cv2.cvtColor(output_image, cv2.COLOR_BGR2HSV)

lower_blue = np.array([100, 50, 50])  
upper_blue = np.array([140, 255, 255]) 

# Create a mask to extract only the blue regions in the image
mask = cv2.inRange(hsv, lower_blue, upper_blue)

# Find contours in the mask
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# List to store the centers of already drawn circles
existing_circle_centers = []


# Draw circles below the detected contours (assuming contours represent players)
for contour in contours:
    # Find the bounding box of the contour
    x, y, w, h = cv2.boundingRect(contour)
    
    M = cv2.moments(contour)
    if M["m00"] != 0:
        cx = int(M["m10"] / M["m00"])
        cy = int(M["m01"] / M["m00"])
        
    # Calculate the center of the bottom edge of the bounding box
    bottom_center = (x + w // 2, y + h)
    
    # Check if the new circle is too close to any existing circle
    if not eliminate_close_features((cx, cy + 30), existing_circle_centers, min_distance=1500):
        # Draw a circle below the bottom center
        cv2.circle(output_image, bottom_center, 30, (0, 0, 255), 600)
        
        # Add the center of the newly drawn circle to the list of existing centers
        existing_circle_centers.append(bottom_center)

# Convert the homography image back to the original format
homography_matrix, _ = cv2.findHomography(dst_points, src_points)

reversed_output_image = cv2.warpPerspective(output_image, homography_matrix, (output_image.shape[1], output_image.shape[0]))

cv2.imwrite("Reversed_player_detected.jpg", reversed_output_image)

Display Width =  800
Display Height =  600
Original Width =  4032
Original Height =  3024
Selected points: [[173.  93.]
 [682. 170.]
 [  9. 304.]
 [682. 457.]]
Selected points (original coordinates): [[ 871.92  468.72]
 [3437.28  856.8 ]
 [  45.36 1532.16]
 [3437.28 2303.28]]


True

In [3]:
# Player detection without homography

import cv2
import numpy as np

# Function to check if a point is too close to any of the existing circles
def eliminate_close_features(new_circle_center, existing_circle_centers, min_distance):
    for center in existing_circle_centers:
        if np.linalg.norm(np.array(new_circle_center) - np.array(center)) < min_distance:
            return True
    return False

input_image = cv2.imread("20230403_132407.jpg")

# Convert the image to HSV color space
hsv = cv2.cvtColor(input_image, cv2.COLOR_BGR2HSV)

lower_blue = np.array([100, 50, 50])  
upper_blue = np.array([140, 255, 255]) 

# Create a mask to extract only the blue regions in the image
mask = cv2.inRange(hsv, lower_blue, upper_blue)

# Find contours in the mask
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# List to store the centers of already drawn circles
existing_circle_centers = []

# Draw circles below the detected contours (assuming contours represent players)
for contour in contours:
    # Find the bounding box of the contour
    x, y, w, h = cv2.boundingRect(contour)
    
    M = cv2.moments(contour)
    if M["m00"] != 0:
        cx = int(M["m10"] / M["m00"])
        cy = int(M["m01"] / M["m00"])
        
    # Calculate the center of the bottom edge of the bounding box
    bottom_center = (x + w // 2, y + h)
    
    # Check if the new circle is too close to any existing circle
    if not eliminate_close_features((cx, cy + 30), existing_circle_centers, min_distance=1000):
        # Draw a circle below the bottom center
        cv2.circle(input_image, bottom_center, 30, (0, 0, 255), 450)
        
        # Add the center of the newly drawn circle to the list of existing centers
        existing_circle_centers.append(bottom_center)
        
cv2.imwrite("Player_detected.jpg", input_image)

True